## 🛠️ **ETL (Extract, Transform, Load)**
### **📂Procesamos el 2do archivo: `user_items.json.gz`**


### 📦 **Extracción  y exploración**

#### **Importamos las librerías que vamos a usar**


In [1]:
import pandas as pd  # Pandas se utiliza para el manejo y análisis de datos tabulares
import pyarrow as pa  # PyArrow se utiliza para trabajar con formatos de datos columnares y eficientes como Parquet
import pyarrow.parquet as pq  # Importamos Parquet
from pandas import json_normalize

import ast  # AST (Abstract Syntax Trees) se utiliza para interpretar expresiones Python
import gzip
import json  # JSON se utiliza para trabajar con datos en formato JSON
import os  # OS proporciona funciones para interactuar con el sistema operativo
import time
import warnings  # Warnings se utiliza para gestionar advertencias y filtrarlas si es necesario
warnings.filterwarnings("ignore")
from data_utils import data_type_check, duplicados_columna
# Autoreload se utiliza para recargar automáticamente los módulos al realizar cambios
%load_ext autoreload
%autoreload 2

#### **Descomprimimos el archivo gz** 

In [2]:
# Obtenemos el tiempo de inicio de todo este ipynb 
start_time = time.time()

def descomprimir_archivos_gz(archivos_gz, carpeta_destino):
    for archivo_gz in archivos_gz:
        with gzip.open(archivo_gz, 'rb') as f_in:
            contenido = f_in.read()
            archivo_destino = os.path.join(carpeta_destino, os.path.splitext(os.path.basename(archivo_gz))[0])
            with open(archivo_destino, 'wb') as f_out:
                f_out.write(contenido)
        print(f'Archivo descomprimido: {archivo_destino}')

# Ejemplo de uso con una lista de archivos gz
archivo_gz_a_descomprimir = ['../0 Dataset/users_items.json.gz']
carpeta_destino = '../0 Dataset/'

descomprimir_archivos_gz(archivo_gz_a_descomprimir, carpeta_destino)

Archivo descomprimido: ../0 Dataset/users_items.json


### **📂Procesamos el 2do archivo: `user_items.json.gz`**


- Tomamos los datos del archivo JSON, transformamos en un DataFrame y realizamos una primera observación de su contenido.

In [3]:
#Creamos una lista vacía llamada "rows" donde almacenaremos los datos del archivo JSON.
rows = []
#Abrir el archivo "user_reviews.json/australian_user_reviews.json" con la codificación MacRoman.
with open("../0 Dataset/users_items.json","r", encoding='utf-8') as f:
    # Leer cada línea del archivo.
    for line in f.readlines():
        # Utilizar "ast.literal_eval" para convertir cada línea en un diccionario de Python
        # y agregarlo a la lista "rows".
        rows.append(ast.literal_eval(line))

#Crear un DataFrame de Pandas a partir de la lista de diccionarios "rows".
df_users_items = pd.DataFrame(rows)
#Veamos unos registros al asar
df_users_items.sample(2)

,user_id,items_count,steam_id,user_url,items
14816,commanderj13,134,76561198058617449,http://steamcommunity.com/id/commanderj13,"[{'item_id': '70', 'item_name': 'Half-Life', '..."
12024,kryzikk,123,76561198074515463,http://steamcommunity.com/id/kryzikk,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


Usando la funcion personalizada `data_type_check` invocada desde `data_utils.py` podemos observar:
- Variables categóricas
- Variables numéricas
- Dimensiones del dataframe
- Nulos
- Tipos de datos
- Informacion acerca de los datos faltantes o nulos de cada columna    


In [4]:
data_type_check(df_users_items)


 Resumen del dataframe:

Dimensiones:  (88310, 5)
       columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0      user_id       100.0      0.0            0    object
1  items_count       100.0      0.0            0     int64
2     steam_id       100.0      0.0            0    object
3     user_url       100.0      0.0            0    object
4        items       100.0      0.0            0    object


Observamos las columnas del dataset y descubrimos que este conjunto contiene 5 columnas y 88310 filas sin nulos.

In [5]:
df_users_items.sample(1)

,user_id,items_count,steam_id,user_url,items
41595,76561198071620258,0,76561198071620258,http://steamcommunity.com/profiles/76561198071...,[]


### 🔁 **TRANSFORM**

#### Tratamiento  de la columna **'items'** (es una lista de diccionarios)
Desanidado y normalizacion de items, que contiene:
- Item_name
- playtime_forever
- playtime_2weeks

Exploramos items para entender su estructura.

In [6]:
df_users_items['items'].sample(5)

54714    [{'item_id': '205790', 'item_name': 'Dota 2 Te...
25843    [{'item_id': '240', 'item_name': 'Counter-Stri...
80077    [{'item_id': '220260', 'item_name': 'Farming S...
42401    [{'item_id': '4000', 'item_name': 'Garry's Mod...
18110    [{'item_id': '220', 'item_name': 'Half-Life 2'...
Name: items, dtype: object

La normalización se lleva a cabo en la columna, descomponiendo el diccionario para obtener una columna distinta por cada clave presente en él. 

In [7]:
#Creamos una nueva fila para cada elemento de la lista en la columna items
df_users_items = df_users_items.explode("items").reset_index()
#Eliminamos la columna index
df_users_items = df_users_items.drop(columns="index")
# Creamos una nueva columna para cada elemento de la lista en la columna items
df_users_items = pd.concat([df_users_items, pd.json_normalize(df_users_items['items'])], axis=1)
#Una vez extraidos eliminamos la columna items
df_users_items.drop(columns=['items'], inplace=True)

In [8]:
data_type_check(df_users_items)


 Resumen del dataframe:

Dimensiones:  (5170015, 8)
            columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0           user_id      100.00     0.00            0    object
1       items_count      100.00     0.00            0     int64
2          steam_id      100.00     0.00            0    object
3          user_url      100.00     0.00            0    object
4           item_id       99.67     0.33        16806    object
5         item_name       99.67     0.33        16806    object
6  playtime_forever       99.67     0.33        16806   float64
7   playtime_2weeks       99.67     0.33        16806   float64


Verificamos si tenemos duplicados en el Dataframe.

In [9]:
# Devuelve duplicados
duplicados = df_users_items.loc[df_users_items.duplicated()]
duplicados

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
75762,bokkkbokkk,0,76561198006988360,http://steamcommunity.com/id/bokkkbokkk,NaN,NaN,NaN,NaN
164518,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,20,Team Fortress Classic,5.0,0.0
164519,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,50,Half-Life: Opposing Force,0.0,0.0
164520,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,70,Half-Life,0.0,0.0
164521,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad,130,Half-Life: Blue Shift,0.0,0.0
...,...,...,...,...,...,...,...,...
4910939,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...,221910,The Stanley Parable,53.0,0.0
4910940,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...,261030,The Walking Dead: Season Two,253.0,0.0
4910941,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...,273110,Counter-Strike Nexon: Zombies,0.0,0.0
4910942,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...,730,Counter-Strike: Global Offensive,0.0,0.0


Contamos 59196 filas con datos duplicados. Procedemos a borrarlos.


In [10]:
df_users_items = df_users_items.drop_duplicates(keep='first')
data_type_check(df_users_items)


 Resumen del dataframe:

Dimensiones:  (5110819, 8)
            columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0           user_id      100.00     0.00            0    object
1       items_count      100.00     0.00            0     int64
2          steam_id      100.00     0.00            0    object
3          user_url      100.00     0.00            0    object
4           item_id       99.67     0.33        16714    object
5         item_name       99.67     0.33        16714    object
6  playtime_forever       99.67     0.33        16714   float64
7   playtime_2weeks       99.67     0.33        16714   float64


#### Playtime_forever: 

Como ya tenemos la informacion de tiempo de juego acumulado por usuario en **playtime_forever**, borramos la columna **playtime_2weeks**


In [11]:
df_users_items = df_users_items.drop('playtime_2weeks', axis=1)

In [12]:
#Verificar duplicados en playtime_forever       
df_users_items.duplicated().sum()

13

In [13]:
# Asignar 0 a los valores nulos en playtime_forever     
df_users_items['playtime_forever'].fillna(0, inplace=True)
data_type_check(df_users_items)


 Resumen del dataframe:

Dimensiones:  (5110819, 7)
            columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0           user_id      100.00     0.00            0    object
1       items_count      100.00     0.00            0     int64
2          steam_id      100.00     0.00            0    object
3          user_url      100.00     0.00            0    object
4           item_id       99.67     0.33        16714    object
5         item_name       99.67     0.33        16714    object
6  playtime_forever      100.00     0.00            0   float64


#### item_name

In [14]:
df_users_items['item_name'].fillna(0, inplace=True)
df_users_items['item_name'] = df_users_items['item_name'].astype(str)

#### steam_id y user_url
- Eliminamos las columnas irrelevantes que no nos riven

In [15]:
df_users_items.drop(["user_url","steam_id"], axis=1, inplace=True)

#### user_id

In [16]:
# Eliminar espacios en blanco al principio y al final de los valores
df_users_items['user_id'] = df_users_items['user_id'].str.strip()
# Llenar valores nulos con un valor específico (por ejemplo, cadena vacía)
df_users_items['user_id'].fillna(' ', inplace=True)
# Eliminar caracteres especiales o no imprimibles
df_users_items['user_id'] = df_users_items['user_id'].str.replace(r'\W', '')
# Convertir a tipo string
df_users_items = df_users_items.astype({'user_id': 'string'})


#### ID

In [17]:
#Borramos nulos en item_id
df_users_items = df_users_items[df_users_items['item_id'].notna()]

### 📤 **LOAD**

In [18]:
#Dejamos informacion de muestra acerca de ese archivo
data_type_check(df_users_items)


 Resumen del dataframe:

Dimensiones:  (5094105, 5)
            columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0           user_id       100.0      0.0            0  string[python]
1       items_count       100.0      0.0            0           int64
2           item_id       100.0      0.0            0          object
3         item_name       100.0      0.0            0          object
4  playtime_forever       100.0      0.0            0         float64


In [19]:
#Guardamos los cambios en parquet
table = pa.Table.from_pandas(df_users_items)
ruta_parquet = os.path.join('..', '0 Dataset', '1.2_users_items_LISTO.parquet')
df_users_items.to_parquet(ruta_parquet)
print(f'Se guardó el archivo {ruta_parquet}')

Se guardó el archivo ..\0 Dataset\1.2_users_items_LISTO.parquet


Eliminamos el archivo descomprimidos que ahora tenemos limpio y liviano en formato parquet

In [20]:
# Lista de archivos descomprimidos
gz_descomprimidos = [
    '../0 Dataset/users_items.json'
]

# Eliminar archivos descomprimidos
for archivo_json in gz_descomprimidos:
    try:
        os.remove(archivo_json)
        print(f'Archivo eliminado: {archivo_json}')
    except FileNotFoundError:
        print(f'Archivo no encontrado: {archivo_json}')

Archivo eliminado: ../0 Dataset/users_items.json


Observamos el tiempo de ejecucion total de nuestro proceso ETL 🔥

In [21]:
# Obtener el tiempo de finalización
end_time = time.time()
# Calcular el tiempo total de ejecución
total_time = end_time - start_time
# Convertir a minutos y redondear a 2 decimales
total_time_minutes = round(total_time / 60, 2)
# Imprimir resultados
print(f"Tiempo total de ejecución de este ipynb: {total_time_minutes} minutos")

Tiempo total de ejecución de este ipynb: 6.3 minutos
